In [1]:
import psycopg2
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("PostgresSQLConn") \
    .config("spark.jars", "/home/prabina-bhushal/jars/postgresql-42.5.6.jar") \
    .getOrCreate()
spark

25/04/29 17:28:45 WARN Utils: Your hostname, prabina-bhushal resolves to a loopback address: 127.0.1.1; using 10.10.42.115 instead (on interface enp2s0)
25/04/29 17:28:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/29 17:28:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
jdbc_url = "jdbc:postgresql://localhost:5432/postgres"
conn_properties = {
    "user" : "postgres",
    "password" : "123",
    "driver" :"org.postgresql.Driver"

}

In [4]:
conn = psycopg2.connect(
    port = 5432,
    host = "localhost",
    dbname='postgres',
    user ="postgres",
    password = "123",
)
cursor = conn.cursor()

In [5]:
provider_table = """
CREATE TABLE IF NOT EXISTS provider_table (
    provider_group_id BIGINT,
    npi BIGINT,
    tin_type SMALLINT,
    tin TEXT 
);
"""
cursor.execute(provider_table)
conn.commit()

In [6]:
spark2 = spark.read.parquet('ProviderScrub.parquet')

In [7]:
spark2.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: short (nullable = true)
 |-- tin: string (nullable = true)



In [8]:
spark2.write.jdbc(url=jdbc_url, table="provider_table", mode="append", properties= conn_properties)

In [15]:
in_network_table = """
CREATE TABLE IF NOT EXISTS in_network (
    billing_code TEXT,
    billing_code_type TEXT,
    negotiation_arrangement TEXT,
    provider_group_id BIGINT,
    negotiated_type TEXT,
    negotiated_rate DOUBLE PRECISION,
    billing_class TEXT,
    billing_code_modifier TEXT[],
    service_code INTEGER[]
);
"""
cursor.execute(in_network_table)
conn.commit()

In [16]:
spark1 = spark.read.parquet('NetworkScrub.parquet')

In [12]:
spark1.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- service_code: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [17]:
spark1.write.jdbc(url=jdbc_url, table="in_network", mode="append", properties=conn_properties)